## Neste código construiremos 3 redes neurais para aproximar derivadas.
## Nos meus testes os resultados foram positivos, mas não posso garantir a consistência dos resultados devido a aleatoriedade do conjunto de dados criado.

In [1]:
import tensorflow as tf
import numpy as np

float_pres='float64'

In [24]:
# Criando conjunto de dados

data_x_list=[]
data_y_list=[]
pi=np.pi


Δx = 0.01                                 # Distância espacial dos pontos na malha utilizada
x = tf.expand_dims(tf.range(-1, 1, Δx, dtype=float_pres),axis=0) # Gerando a malha de pontos no espaço unidimensional

for i in range(5000):
    
    # Gerando uma condição inicial aleatória
    #------------------------------------------------------------------------------------------------------------------
    k1 = tf.random.uniform([1], 0, 5, dtype='int32')   # Amostrando uma frequência aleatória para a função seno
    k1 = tf.cast(k1, dtype=float_pres)                  # Mudando o tipo do tensor
    a  = tf.random.uniform([1], 0, 1, dtype=float_pres) # Amostrando um peso aleatória para ponderar as funções seno
    #------------------------------------------------------------------------------------------------------------------
    
    # Valor da função
    u =     a * tf.math.sin(k1*pi*x)
    # Valor da derivada
    du= a*k1*pi*tf.math.cos(k1*pi*x)

    
    data_x_list.append(u)
    data_y_list.append((u[:,:-1]-u[:,1:])/Δx)
    
    # Gerando uma condição inicial aleatória
    #------------------------------------------------------------------------------------------------------------------
    k = tf.random.normal([1], 0, np.sqrt(20),dtype=float_pres)
    #------------------------------------------------------------------------------------------------------------------
    
    # Valor da função
    u = x*0 + k
    # Valor da derivada
    du= x*0

    
    data_x_list.append(u)
    data_y_list.append((u[:,:-1]-u[:,1:])/Δx)
    
    # Gerando uma condição inicial aleatória
    #------------------------------------------------------------------------------------------------------------------
    a = tf.random.normal([1], 0, np.sqrt(20),dtype=float_pres)
    k = tf.random.normal([1], 0, np.sqrt(20),dtype=float_pres)
    #------------------------------------------------------------------------------------------------------------------
    
    # Valor da função
    u = a*x+k
    # Valor da derivada
    du= x*0+a

    
    data_x_list.append(u)
    data_y_list.append((u[:,1:]-u[:,:-1])/Δx)
    
data_x=tf.concat(data_x_list,axis=0)
data_y=tf.concat(data_y_list,axis=0)

train_x=data_x[:int(15000*0.8)]
train_y=data_y[:int(15000*0.8)]
test_x=data_x[int(15000*0.8):]
test_y=data_y[int(15000*0.8):]

In [29]:
train_y.shape

TensorShape([12000, 199])

In [30]:
# # Criando um modelo sequencial
# model_foward=tf.keras.models.Sequential()

# # Adicionando camadas
# model_foward.add(tf.keras.layers.Reshape([400,1],dtype=float_pres)) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
#                                             # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
# model_foward.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=2, use_bias=False,dtype=float_pres)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 399.
#                                                                                                        # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
#                                                                                                        # Então o output da convolução será: n x (s-a+1) x b
#                                                                                                        # Dada a natureza do problema, sabemos que não é necessário um bias (use_bias=False), então removemos ele para evitar overfitting.
# model_foward.add(tf.keras.layers.Flatten(dtype=float_pres)) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


# optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
# model_foward.compile(loss='mean_squared_error',optimizer=optimizer)

In [31]:
input_y=tf.keras.layers.Input([200],dtype='float64')
layers=[]
layers.append(tf.keras.layers.Reshape([200,1],dtype=float_pres))
layers.append(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=2, use_bias=False,dtype=float_pres))
layers.append(tf.keras.layers.Flatten(dtype=float_pres))

output_y=input_y/Δx

for layer in layers:
    output_y=layer(output_y)

model_foward=tf.keras.Model(input_y, output_y)

optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_foward.compile(loss='mean_squared_error',optimizer=optimizer)

In [32]:
history = model_foward.fit(train_x, train_y, # Como estamos calculando a derivada "pra frente", devemos remover a última derivada
                           batch_size=16,
                           epochs=50,
                           validation_split=0.2)
test_scores = model_foward.evaluate(test_x, test_y[:,:-1], verbose=1)
print('Test loss:', test_scores)

Epoch 1/50
600/600 [==============================] - 4s 3ms/step - loss: 809.3066 - val_loss: 1084.9838
Epoch 2/50
600/600 [==============================] - 2s 3ms/step - loss: 795.6268 - val_loss: 1084.4694
Epoch 3/50
600/600 [==============================] - 2s 4ms/step - loss: 788.9760 - val_loss: 1084.0786
Epoch 4/50
600/600 [==============================] - 2s 4ms/step - loss: 766.9370 - val_loss: 1083.5223
Epoch 5/50
600/600 [==============================] - 2s 3ms/step - loss: 786.0712 - val_loss: 1083.1025
Epoch 6/50
600/600 [==============================] - 2s 3ms/step - loss: 789.8687 - val_loss: 1082.3237
Epoch 7/50
600/600 [==============================] - 2s 3ms/step - loss: 785.3183 - val_loss: 1081.7872
Epoch 8/50
600/600 [==============================] - 2s 3ms/step - loss: 781.4075 - val_loss: 1081.5962
Epoch 9/50
600/600 [==============================] - 2s 3ms/step - loss: 792.4211 - val_loss: 1081.0580
Epoch 10/50
600/600 [==============================] - 

ValueError: in user code:

    C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py:1330 test_function  *
        return step_function(self, iterator)
    C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py:1320 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Miniconda3\lib\site-packages\keras\engine\training.py:1313 run_step  **
        outputs = model.test_step(data)
    C:\

In [33]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_foward.layers[3].weights[0]

<tf.Variable 'conv1d_4/kernel:0' shape=(2, 1, 1) dtype=float64, numpy=
array([[[ 0.89777252]],

       [[-0.98328734]]])>

In [97]:
# Coeficientes exatos
print(tf.math.reduce_mean(((test_x[:,1:]-test_x[:,:-1])/Δx-test_y[:,:-1])**2))

# Outros coeficientes
print(model_foward.evaluate(test_x,test_y[:,:-1]))

tf.Tensor(0.015893298385611498, shape=(), dtype=float64)
32/32 [==============================] - 0s 2ms/step - loss: 117.7257
117.72566223144531


In [14]:
# Criando um modelo sequencial
model_backward=tf.keras.models.Sequential()

# Adicionando camadas
model_backward.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
model_backward.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=2, use_bias=False)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 399.
                                                                                                         # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                         # Então o output da convolução será: n x (s-a+1) x b
model_backward.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_backward.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_squared_error'])

In [ ]:
history = model_backward.fit(train_x, train_y[:,1:], # Como estamos calculando a derivada "pra trás", devemos remover a primeira derivada
                    batch_size=512,
                    epochs=30,
                    validation_split=0.2)
test_scores = model_backward.evaluate(test_x, test_y[:,1:], verbose=1)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

In [ ]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_backward.layers[1].weights[0]*Δx

In [ ]:
# Criando um modelo sequencial
model_middle=tf.keras.models.Sequential()

# Adicionando camadas
model_middle.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
# Observe que, para a derivada centrada, o kernel_size deve ser 3.
model_middle.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=3, use_bias=False)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 398.
                                                                                                       # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                       # Então o output da convolução será: n x (s-a+1) x b
model_middle.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_middle.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_squared_error'])

In [ ]:
history = model_middle.fit(train_x, train_y[:,1:-1], # Como estamos calculando a derivada centrada, devemos remover a primeira e a última derivada
                    batch_size=512,
                    epochs=30,
                    validation_split=0.2)
test_scores = model_middle.evaluate(test_x, test_y[:,1:-1], verbose=1)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

In [ ]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_middle.layers[1].weights[0]*Δx